In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib widget

In [ ]:
from widget_groove_sel import GrooveSelectionWidget, WidgetGrooveSelectionTCPMovement

In [ ]:
#%time g = GrooveSelectionWidget()

In [ ]:
#%time g.display()

In [ ]:
sel_movement = WidgetGrooveSelectionTCPMovement()
sel_movement.display()